# Required modules
- PyYAML
- pyorient

In [25]:
from appconfig import AppConfig, DataFileContainer
from stockdatabase import StockDatabase
import pandas as pd
import os

In [4]:
def batch_insert_index_values_from_files(database, index_file_folder_path, file_list):
    print("-- Inserting index values")
    current_index = 1
    for file in file_list:
        path = os.path.join(index_file_folder_path, file["file_name"])
        df = pd.read_csv(path)
        print("Inserting '" + file["file_name"] + "' (" + str(current_index) +
              " of " + str(len(file_list)) + ") (" + str(df.shape[0]) + " rows) ...")
        current_index += 1
        database.insert_index_values(df, file["isin"], file["region"])
    print("-- Done!")

# Load information from config files

In [5]:
config = AppConfig("config/config.yml")
data_file_list = DataFileContainer("config/files.yml")

# Setup database (connect and create table and primary key fields)

In [6]:
database = StockDatabase(config.database_user, config.database_password, config.database_name,
                         config.database_host, config.database_port, config.table_index_values_name)
database.connect()
database.recreate_tables()

# Insert all index values from csv files

In [7]:
batch_insert_index_values_from_files(database, config.data_file_folder, data_file_list.index_value_files)

-- Inserting index values
Inserting 'Index CAC40 (FCHI) Entwicklung.csv' (1 of 7) (8050 rows) ...
Inserting 'Index DAX Entwicklung.csv' (2 of 7) (8611 rows) ...
Inserting 'Index Dow Jones (DJI) Entwicklung.csv' (3 of 7) (5346 rows) ...
Inserting 'Index Euronext 100 (N100) Entwicklung.csv' (4 of 7) (5484 rows) ...
Inserting 'Index NASDAQ100 (NDX) Entwicklung.csv' (5 of 7) (5366 rows) ...
Inserting 'Index S&P500 Entwicklung.csv' (6 of 7) (5366 rows) ...
Inserting 'Index STOXX50E Entwicklung seit 2007.csv' (7 of 7) (3540 rows) ...
-- Done!


# Test query (check if any values in database)

In [28]:
print(database.query_index_values("DE0008469008", limit=1)[0].__dict__)

{'date': datetime.date(1987, 12, 30), 'open': 1005.190002, 'high': 1005.190002, 'low': 1005.190002, 'close': 1005.190002, 'adj_close': 1005.190002, 'volume': 0.0, 'isin': 'DE0008469008', 'region': 'EU'}
